In [21]:
# 마지막으로 손으로 직접 잘못된 영화코드 수정한 데이터프레임 : code_added_df.csv

In [32]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from selenium import webdriver
import re
import matplotlib as mpl
from matplotlib import rc
import missingno as msno
import warnings
import datetime
from scrapy.http import TextResponse
warnings.filterwarnings('ignore')

# matplotlib 한글 사용을 위한 코드
rc('font', family='AppleGothic')

In [34]:
final_df = pd.read_csv('./final_df_03_ols.csv').drop(columns=['Unnamed: 0'])

In [27]:
df = pd.read_csv('./../code_added_df.csv').drop(columns=['Unnamed: 0'])
df.head(1)

,id,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분,N_movie_code
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,1843,88582586366,10494499,25001906266,2823388,액션,12세이상관람가,일반영화,98438


In [38]:
df.iloc[338]

id                     338
영화명                      빅
감독                   트레버 월
제작사                    NaN
수입사               (주)누리픽쳐스
배급사               (주)이수C&E
개봉일             2016-07-27
영화유형                  개봉영화
영화형태                    장편
국적                      미국
전국스크린수                 332
전국매출액           1174196720
전국관객수               160860
서울매출액            215962400
서울관객수                28656
장르                   애니메이션
등급                   전체관람가
영화구분               독립/예술영화
N_movie_code         96379
Name: 338, dtype: object

## 진짜 마지막 수정
- 러닝타임은 잘못된거 직접 수정하기

In [130]:
# column : audience_rating/expert_rating, index : 168  '맥베스' 
# column : audience_rating/expert_rating index : 338 '빅'  
# column : audience_rating, index : 902 '원더랜드'



In [65]:
false_index = [403]  # 이 영화들은 다른 정보들도 바꿔야함
false_N_code = [152168]

In [147]:
final_df.iloc[403]

id                              403
영화명                              하루
감독                              조선호
제작사                         (주)라인필름
수입사                             NaN
배급사                        CGV아트하우스
개봉일                      2017-06-15
영화유형                           개봉영화
영화형태                             장편
국적                               한국
전국스크린수                          781
전국매출액                    8721627698
전국관객수                       1085087
서울매출액                    2165557498
서울관객수                        260218
장르                              스릴러
등급                         15세이상관람가
영화구분                           일반영화
expert_rating                   4.6
starring_actors       김영민, 변요한, 유재명
running_times                    90
audience_rating                  10
rating_audiences                  1
개봉일_int                        2017
director_power                    0
director_movie_num            1 [1]
Name: 403, dtype: object

In [68]:
# for each_row in df.iterrows():
#     print(each_row[1]['id'])

In [128]:
# final_df['starring_actors'][338] = '롭 슈나이더, 헤더 그레이엄, 켄 정'

In [150]:
final_df.iloc[403]

id                              403
영화명                              하루
감독                              조선호
제작사                         (주)라인필름
수입사                             NaN
배급사                        CGV아트하우스
개봉일                      2017-06-15
영화유형                           개봉영화
영화형태                             장편
국적                               한국
전국스크린수                          781
전국매출액                    8721627698
전국관객수                       1085087
서울매출액                    2165557498
서울관객수                        260218
장르                              스릴러
등급                         15세이상관람가
영화구분                           일반영화
expert_rating                   4.6
starring_actors       김영민, 변요한, 유재명
running_times                    90
audience_rating                  10
rating_audiences                  1
개봉일_int                        2017
director_power                    0
director_movie_num            1 [1]
Name: 403, dtype: object

In [ ]:
# final_df의 특정 row에 대한 평점 값 바꿔치기 하는 코드 
# 데이터프레임의 index로는 false_index를 차례대로 사용한다

In [148]:
index = 0

In [149]:
for each_row in final_df.iterrows():

    
    if int(each_row[1]['id']) in false_index:

        # 먼저, 평점의 가장 마지막 페이지에 해당하는 url의 document구조를 가져온다
        code = int(false_N_code[index])
        num = 99999
        url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, num)

        # 현재 페이지 알아내기
        try:
            req = requests.get(url)
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            number_list = response.xpath('/html/body/div/div/div[6]/div/a/span/text()').extract()
            last_number = number_list[len(number_list) - 1]

            cur_page = int(last_number.replace(',', ''))
            cur_page

            # 평가의 마지막 페이지 마지막 관객의 평점올린 날짜를 가져와서 7주일 이후의 날짜(limit_date) 만들기
            time_elements = response.xpath('/html/body/div/div/div[5]/ul/li/div/dl/dt/em[2]/text()')
            start_date = time_elements[len(time_elements) - 1].extract().split(' ')[0]
            start_date = start_date.split(" ")[0].split('.')
            limit_date = datetime.datetime(int(start_date[0]), int(start_date[1]), int(start_date[2])) + datetime.timedelta(weeks=1)

            audience_number = 0
            total_rating = 0
            flag = 0

            # 여기서부터 정확한 페이지로 크롤링 시작
            for page in range(cur_page, 0, -1):
                if flag == 1:
                    break

                url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, page)
                req = requests.get(url)
                response = TextResponse(req.url, body=req.text, encoding="utf-8")

                rating_elements = response.xpath('/html/body/div/div/div[5]/ul/li')

                # 평가의 해당 페이지에서, 각 평점 엘리먼트 하나마다 날짜를 확인하면서 limit을 넘겼는지 체크한다
                for n in range(len(rating_elements), 0, -1):
                    star = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[1]/em/text()'.format(n)).extract()[0]
                    cur_date = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[2]/dl/dt/em[2]/text()'.format(n)).extract()[0].split(' ')[0].split('.')

                    check_limit = limit_date - datetime.datetime(int(cur_date[0]), int(cur_date[1]), int(cur_date[2]))
                    if check_limit.days < 0:
                        flag = 1
                        break

                    total_rating = total_rating + int(star)
                    audience_number += 1

            final_df['audience_rating'][false_index[index]] = total_rating/audience_number
            final_df['rating_audiences'][false_index[index]] = audience_number            
        except Exception:
            final_df['audience_rating'][false_index[index]] = 0
            final_df['rating_audiences'][false_index[index]] = 0
            
        index += 1

    
    

In [100]:
index

3

In [132]:
# final_df.to_csv('final_df_04.csv')

## 러닝타임 고치기

In [ ]:
# column : running_times, index : 338
# column : running_times, index : 902
# 러닝타임 0인 영화 : 338 빅(88분), 902원더랜드(85분)
# 러닝타임 20인 영화 : 403 하루 (90분)

In [151]:
final_df.iloc[338]

id                                     338
영화명                                      빅
감독                                   트레버 월
제작사                                    NaN
수입사                               (주)누리픽쳐스
배급사                               (주)이수C&E
개봉일                             2016-07-27
영화유형                                  개봉영화
영화형태                                    장편
국적                                      미국
전국스크린수                                 332
전국매출액                           1174196720
전국관객수                               160860
서울매출액                            215962400
서울관객수                                28656
장르                                   애니메이션
등급                                   전체관람가
영화구분                               독립/예술영화
expert_rating                            0
starring_actors       롭 슈나이더, 헤더 그레이엄, 켄 정
running_times                           91
audience_rating                     8.2462
rating_audiences                       329
개봉일_int    

In [144]:
final_df['expert_rating'][403] = float(4.60)

In [153]:
final_df.to_csv('final_df_04.csv')

## 배우 동명이인 처리하고, 배우 영향력 점수 추가

In [168]:
# 배우 동명이인 처리하고 영향력 칼럼 추가할 차례임//  2020.03.28

# 이광수B	149	포켓몬 더 무비 XY 후파: 광륜의 초마신	
# 이광수A	166	돌연변이
# 이광수A	571	탐정: 리턴즈
# 이광수A	765	나의 특별한 형제
# 김태균A 341 장난감이 살아있다
# 김태균B 452 슈퍼빼꼼: 스파이 대작전 
# 소연A 419 터닝메카드W: 블랙미러의 부활
# 소연B 814 교회오빠
# 소연A 895 극장판 엉덩이 탐정: 화려한 사건 수첩
# 민아A 554 래빗 스쿨
# 민아B 636 번개맨과 신비의 섬
# 민아A 638 매직빈

In [166]:
final_df.iloc[149]

id                                                                  149
영화명                                             포켓몬 더 무비 XY 후파: 광륜의 초마신
감독                                                             유야마 쿠니히코
제작사                                                                 NaN
수입사                                                           (주)포켓몬코리아
배급사                                                            (주)이수C&E
개봉일                                                          2015-12-23
영화유형                                                               개봉영화
영화형태                                                                 장편
국적                                                                   일본
전국스크린수                                                              360
전국매출액                                                        1528041500
전국관객수                                                            214284
서울매출액                                                         29

In [167]:
actor_compare_df

,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,...,제작사,배급사,등급,장르,감독,배우,main_actors,개봉일_int,actor_involved,전국관객수
0,어벤져스,2012-04-26,59546521565,0.093,59546521565,7073037,7073037,967,109646,미국,...,NaN,한국소니픽쳐스릴리징브에나비스타영화㈜,12세이상관람가,"액션,SF",조스 웨던,"로버트 다우니 주니어,스칼렛 요한슨,크리스 헴스워스,크리스 에반스,제레미 레너","로버트 다우니 주니어, 스칼렛 요한슨, 크리스 헴스워스",2012,True,7073037
1,범죄와의 전쟁: 나쁜놈들 전성시대,2012-02-02,36538823500,0.057,36538823500,4719872,4719872,713,91889,한국,...,"(주)팔레트픽처스,(주)쇼박스",(주)쇼박스,청소년관람불가,"범죄,드라마",윤종빈,"최민식,하정우,조진웅,마동석,곽도원,김성균,박성광,이철민,임자연,임헌호,권종현,김민...","최민식, 하정우, 조진웅",2012,True,4710454
2,내 아내의 모든 것,2012-05-17,32518413000,0.051,32518413000,4360205,4360205,563,82867,한국,...,"(주)수필름,영화사 집",(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"코미디,멜로/로맨스",민규동,"임수정,이선균,류승룡,이달형,이진희,김도균,김선일,최차원,신경철,옥광희,박희본,김지...","임수정, 이선균, 류승룡",2012,True,4360205
3,댄싱퀸,2012-01-18,30129331500,0.047,30129331500,4057555,4057555,544,80969,한국,...,"(주)제이케이필름,씨제이이앤엠(주)",씨제이이앤엠(주),12세이상관람가,"코미디,드라마",이석훈,"황정민,엄정화,정성화,오나라,박정민,박사랑,임호걸,정원영,이정건,안은정,라미란,송원...","황정민, 엄정화, 정성화",2012,True,4057546
4,부러진 화살,2012-01-18,25878110500,0.040,25936754000,3450941,3459780,530,71556,한국,...,(주)아우라픽처스,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,드라마,정지영,"안성기,박원상,나영희,김지호,이승훈,이춘연,이홍표,임태연,장길수,장현남,전윤희,조세...","안성기, 박원상, 나영희",2012,True,3450941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,앳 퍼스트 라이트,2019-09-25,650700,0.000,650700,89,89,8,49,캐나다,...,NaN,이언픽쳐스,12세이상관람가,"드라마,SF,스릴러",제이슨 스톤,"스테파니 스콧,테오도르 펠르랭,세이드 타그마오우이","스테파니 스콧, 테오도르 펠르랭, 세이드 타그마오우이",2019,True,89
1667,액츠 오브 바이올런스,2019-07-15,403000,0.000,403000,71,71,20,71,미국,...,NaN,(주)누리픽쳐스,청소년관람불가,"액션,드라마",브렛 도노후,"브루스 윌리스,콜 하우저,애쉬튼 홈즈,멜리사 볼로나,숀 애쉬모어","브루스 윌리스, 콜 하우저, 애쉬튼 홈즈",2019,True,71
1668,먹거나 먹히거나,2019-10-10,44000,0.000,44000,18,18,5,18,미국,...,NaN,㈜코리아스크린,청소년관람불가,"액션,범죄,드라마",폴 슈레이더,"니콜라스 케이지,윌렘 데포,루이자 크로즈,폴 슈레이더","니콜라스 케이지, 윌렘 데포, 루이자 크로즈",2019,True,18
1669,백트레이스,2019-07-30,1000,0.000,1000,1,1,1,1,미국,...,NaN,주식회사 풍경소리,15세이상관람가,"액션,스릴러",브라이언 A 밀러,"실베스터 스탤론,라이언 구즈만,매튜 모딘","실베스터 스탤론, 라이언 구즈만, 매튜 모딘",2019,True,1


In [161]:
actor_compare_df = pd.read_csv('../actor_compare_df_sep_d.csv').drop(columns=['Unnamed: 0'])

In [164]:
actor_compare_df.head(1)

,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,...,제작사,배급사,등급,장르,감독,배우,main_actors,개봉일_int,actor_involved,전국관객수
0,어벤져스,2012-04-26,59546521565,0.093,59546521565,7073037,7073037,967,109646,미국,...,NaN,한국소니픽쳐스릴리징브에나비스타영화㈜,12세이상관람가,"액션,SF",조스 웨던,"로버트 다우니 주니어,스칼렛 요한슨,크리스 헴스워스,크리스 에반스,제레미 레너","로버트 다우니 주니어, 스칼렛 요한슨, 크리스 헴스워스",2012,True,7073037


In [154]:
actor_power_dict = {}
actor_movies_dict = {}

In [155]:
# 배우 성과측정 함수

year_list = list(final_df['개봉일_int'])
year_index = 0

def actor_func(actors):
    global year_index
    global actor_power_dict
    global actor_movies_dict
    
    total_audience = 0
    
    actor_list = actors.split(',')
    actor_list = [re.sub('[(].+[)]', '', each_name) for each_name in actor_list]
    
    limit_year = int(year_list[year_index][:4]) - 3
    
    for each_name in actor_list:
        stripped_name = each_name.strip()
        
        # 데이터프레임 자르기
        cut_movie_df = actor_compare_df[actor_compare_df['개봉일_int'] > limit_year - 1]
        cut_movie_df = cut_movie_df[cut_movie_df['개봉일_int'] < int(year_list[year_index]) + 1]
        
        for i in range(0, len(cut_movie_df)):
            
            cur_row = cut_movie_df.iloc[i]
            cur_row_actors = [name.strip() for name in cur_row['main_actors'].split(',')]
            
            if stripped_name in cur_row_actors:
                total_audience += int(cur_row['전국관객수'])
                
        try:
            actor_power_dict[stripped_name] += total_audience
            actor_movies_dict[stripped_name] += 1
        except Exception:
            actor_power_dict[stripped_name] = total_audience
            actor_movies_dict[stripped_name] = 1
        
    year_index += 1
        
    

In [ ]:
# final_df['starring_actors'].apply(actor_func)

In [ ]:
# final_df['actor_power'] = final_df['starring_actors'].apply(actor_func)

In [165]:
final_df.head(1)

,id,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,...,등급,영화구분,expert_rating,starring_actors,running_times,audience_rating,rating_audiences,개봉일_int,director_power,director_movie_num
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,...,12세이상관람가,일반영화,6.92,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 헴스워스(토르), 마크 러...",141,8.193182,88,2015,17567536.0,1 [1]
